In [1]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

im1 = cv2.imread('/content/drive/My Drive/CVG/monkey_distorted.jpg')          # Image that needs to be registered.
im2 = cv2.imread('/content/drive/My Drive/CVG/monkey.jpg') # trainImage

In [3]:
img1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)


In [4]:
# Initiate ORB detector
orb = cv2.ORB_create(50)  #Registration works with at least 50 points

# find the keypoints and descriptors with orb
kp1, des1 = orb.detectAndCompute(img1, None)  #kp1 --> list of keypoints
kp2, des2 = orb.detectAndCompute(img2, None)

In [5]:
#Brute-Force matcher takes the descriptor of one feature in first set and is 
#matched with all other features in second set using some distance calculation.
# create Matcher object

In [6]:
matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)

In [7]:
# Match descriptors.
matches = matcher.match(des1, des2, None)  #Creates a list of all matches, just like keypoints

# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)

In [8]:
img3 = cv2.drawMatches(im1,kp1, im2, kp2, matches[:10], None)
"""
cv2.imshow("Matches image", img3)
cv2.waitKey(0)
"""

'\ncv2.imshow("Matches image", img3)\ncv2.waitKey(0)\n'

In [9]:
#Now let us use these key points to register two images. 
#Can be used for distortion correction or alignment
#For this task we will use homography. 


# Extract location of good matches.
# For this we will use RANSAC.
#RANSAC is abbreviation of RANdom SAmple Consensus, 
#in summary it can be considered as outlier rejection method for keypoints.
#RANSAC needs all key points indexed, first set indexed to queryIdx
#Second set to #trainIdx. 


In [10]:
points1 = np.zeros((len(matches), 2), dtype=np.float32)  #Prints empty array of size equal to (matches, 2)
points2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches):
   points1[i, :] = kp1[match.queryIdx].pt    #gives index of the descriptor in the list of query descriptors
   points2[i, :] = kp2[match.trainIdx].pt    #gives index of the descriptor in the list of train descriptors


In [11]:
#Now we have all good keypoints so we are ready for homography.   
# Find homography
#https://en.wikipedia.org/wiki/Homography_(computer_vision)
  
h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
 
  # Use homography
height, width, channels = im2.shape
im1Reg = cv2.warpPerspective(im1, h, (width, height))  #Applies a perspective transformation to an image.

In [12]:
print("Estimated homography : \n",  h)

Estimated homography : 
 [[ 1.58086690e+00  1.32767006e-01 -6.29327345e+01]
 [-1.47000605e-02  1.02210326e+00  6.27648927e+00]
 [ 1.84423603e-04  1.14677972e-04  1.00000000e+00]]


In [13]:
"""
cv2.imshow("Registered image", im1Reg)
cv2.waitKey()
"""

'\ncv2.imshow("Registered image", im1Reg)\ncv2.waitKey()\n'